Q1 - The Great Data Merge Maze¶

Question: Welcome to the Great Data Merge Maze!
You are given three datasets: customers, orders, and products.
Your task is to merge these datasets to answer the following questions:

- Which customers ordered which products and at what price?
- Calculate the total amount spent by each customer.
- Identify the top 2 customers who spent the most.
- Determine which products were never ordered.
- Find the customer who ordered the highest quantity of a single product.

Datasets:

customers: Contains customer IDs and names.

orders: Contains order IDs, customer IDs, product IDs, and order quantities.

products: Contains product IDs, names, and prices.

Generate synthetic data for the datasets and merge them to find the answers.

In [ ]:
import pandas as pd
import numpy as np

# Seed for reproducibility
np.random.seed(0)

# Customers DataFrame
customers = pd.DataFrame({
    'customer_id': range(1, 11),
    'customer_name': ['Alice Apples', 'Bob Bananas', 'Charlie Cherries', 'David Dates', 'Eve Elderberries', 'Frank Figs', 'Grace Grapes', 'Hannah Honeydew', 'Ivy Iceberg', 'Jack Jicama']
})

# Products DataFrame
products = pd.DataFrame({
    'product_id': range(1, 11),
    'product_name': ['Widget Wonder', 'Gizmo Glitz', 'Doodad Delight', 'Thingamajig Thrill', 'Contraption Charm', 'Gadget Glow', 'Whatchamacallit Whimsy', 'Doohickey Dazzle', 'Whatsit Whiz', 'Gubbins Galore'],
    'product_price': np.random.uniform(10, 100, size=10).round(2)
})

# Orders DataFrame
orders = pd.DataFrame({
    'order_id': range(1, 21),
    'customer_id': np.random.choice(customers['customer_id'], size=20),
    'product_id': np.random.choice(products['product_id'], size=20),
    'order_quantity': np.random.randint(1, 10, size=20)
})

# Display the datasets
customers.head()


In [ ]:
products.head()

In [ ]:
orders.head()

In [ ]:
cust_orders = pd.merge(customers, orders, how="outer", left_on='customer_id', right_on='customer_id')
cust_orders = pd.merge(cust_orders, products, how="inner", left_on='product_id', right_on='product_id')

In [ ]:
# Which customers ordered which products and at what price?
customer_orders = cust_orders.copy(deep=True)
customer_orders.drop(['customer_id', 'order_id', 'product_id', 'order_quantity'], axis='columns', inplace=True)
customer_orders.set_index(['customer_name'])
customer_orders.head()

In [ ]:
# Calculate the total amount spent by each customer
customer_spending = cust_orders.copy(deep=True)
customer_spending['amount_spent'] = round(customer_spending['order_quantity'] * customer_spending['product_price'], 3)
customer_spending.drop(['customer_id', 'order_id', 'product_id', 'order_quantity', 'product_name', 'product_price'], axis='columns', inplace=True)
customer_spending = customer_spending.groupby('customer_name').aggregate('sum')
customer_spending.head()

In [ ]:
# Identify the top 2 customers who spent the most.
top_customers = customer_spending.sort_values(by='amount_spent', ascending=False).head(2)
top_customers

In [ ]:
# Determine which products were never ordered.
unpopular_products = pd.merge(left=products, right=orders, left_on='product_id', right_on='product_id', how='left')
unpopular_products[unpopular_products['order_quantity'].isnull()].head()

In [ ]:
# Find the customer who ordered the highest quantity of a single product.
star_customer = cust_orders.loc[cust_orders['order_quantity'].idxmax()]
star_customer[['customer_name', 'product_name', 'order_quantity']]